In [1]:
import os
from typing import Any, Dict, List, Optional

import requests
import pandas as pd

# ---- constants
MERKL_API = "https://api.merkl.xyz"
CHAIN_ID = 9745  # Plasma (from your URL)
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# ---- optional price config (kept off by default)
PRICE_CONFIG = {
    "enable_usd": False,
    "enable_xpl": False,
    "manual_token_usd": {},
    "manual_xpl_usd": None,  # e.g., 0.45 -> $/XPL
}


In [2]:
def _get_json(url: str, params: Dict[str, Any] = None, timeout: int = 30):
    r = requests.get(url, params=params or {}, timeout=timeout)
    r.raise_for_status()
    return r.json()

def _safe_float(x) -> Optional[float]:
    try:
        return float(x)
    except Exception:
        return None


In [3]:
def fetch_campaigns_v3_flat(chain_id: int) -> pd.DataFrame:
    """
    Handles the nested shape:
      { "<chainId>": { "<rewardTokenAddress>": { "<campaignId>": { ... } } } }
    Falls back to older shapes if needed.
    """
    payload = _get_json(f"{MERKL_API}/v3/campaigns", params={"chainIds": chain_id})

    rows = []
    chain_key = str(chain_id)
    if isinstance(payload, dict) and chain_key in payload and isinstance(payload[chain_key], dict):
        for reward_token_addr, by_campaign in payload[chain_key].items():
            if not isinstance(by_campaign, dict):
                continue
            for campaign_id, obj in by_campaign.items():
                if not isinstance(obj, dict):
                    continue
                rec = {
                    "campaignId": str(campaign_id),
                    "rewardToken": str(reward_token_addr),
                    "chainId": chain_id,
                }
                rec.update(obj)  # bring through campaignType, amount, amountDecimal, start/end, etc.
                rows.append(rec)
    else:
        # tolerant fallback
        if isinstance(payload, list):
            rows = payload
        elif isinstance(payload, dict):
            for k in ("campaigns", "data", "result", "items"):
                if isinstance(payload.get(k), list):
                    rows = payload[k]
                    break

    df = pd.DataFrame(rows)

    # normalize expected columns
    for col in [
        "campaignId","campaignType","campaignSubType","rewardToken","rewardTokenSymbol",
        "amount","amountDecimal","startTimestamp","endTimestamp","mainParameter"
    ]:
        if col not in df.columns:
            df[col] = None

    # deep link
    df["appLink"] = df["campaignId"].apply(lambda cid: f"https://app.merkl.xyz/?chain={chain_id}#{cid}")

    # numeric coercions where applicable
    df["amountDecimal"] = pd.to_numeric(df["amountDecimal"], errors="coerce")

    return df


In [4]:
# Requires: pip install web3
from web3 import Web3

# RPC_URL = os.getenv("XPL_RPC_URL", "")  # set env var or paste your RPC URL here
RPC_URL = 'https://rpc.plasma.to'
ERC20_ABI_MIN = [
    {"constant":True,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"},
    {"constant":True,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"type":"function"},
    {"constant":True,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"type":"function"},
]

def enrich_with_onchain_token_meta(df: pd.DataFrame, rpc_url: str) -> pd.DataFrame:
    if not rpc_url:
        print("RPC_URL not set; skipping on-chain enrichment.")
        return df

    w3 = Web3(Web3.HTTPProvider(rpc_url, request_kwargs={"timeout": 30}))

    def fetch_meta(addr: str):
        try:
            c = w3.eth.contract(address=Web3.to_checksum_address(addr), abi=ERC20_ABI_MIN)
            decimals = c.functions.decimals().call()
            try: symbol = c.functions.symbol().call()
            except: symbol = None
            try: name = c.functions.name().call()
            except: name = None
            return decimals, symbol, name
        except Exception:
            return None, None, None

    addrs = (
        df["rewardToken"]
        .dropna()
        .astype(str)
        .str.lower()
        .drop_duplicates()
        .tolist()
    )

    meta = {}
    for a in addrs:
        d, s, n = fetch_meta(a)
        meta[a] = {"decimals": d, "symbol_onchain": s, "name_onchain": n}

    out = df.copy()
    out["rewardToken_lc"] = out["rewardToken"].astype(str).str.lower()
    out["tokenDecimals"] = out["rewardToken_lc"].map(lambda x: meta.get(x, {}).get("decimals"))
    out["symbol_onchain"] = out["rewardToken_lc"].map(lambda x: meta.get(x, {}).get("symbol_onchain"))
    out["name_onchain"] = out["rewardToken_lc"].map(lambda x: meta.get(x, {}).get("name_onchain"))

    # compute amountDecimal if missing and we have decimals
    mask = out["amountDecimal"].isna() & out["amount"].notna() & out["tokenDecimals"].notna()
    out.loc[mask, "amountDecimal"] = (
        pd.to_numeric(out.loc[mask, "amount"], errors="coerce") /
        (10 ** out.loc[mask, "tokenDecimals"].astype(int))
    )

    # prefer on-chain symbol when API didn't provide one
    out["rewardTokenSymbol"] = out["rewardTokenSymbol"].fillna(out["symbol_onchain"])
    return out.drop(columns=["rewardToken_lc"])


In [5]:
# Requires: pip install web3
from web3 import Web3

# RPC_URL = os.getenv("XPL_RPC_URL", "")  # set env var or paste your RPC URL here
RPC_URL = 'https://rpc.plasma.to'
ERC20_ABI_MIN = [
    {"constant":True,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"},
    {"constant":True,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"type":"function"},
    {"constant":True,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"type":"function"},
]

def enrich_with_onchain_token_meta(df: pd.DataFrame, rpc_url: str) -> pd.DataFrame:
    if not rpc_url:
        print("RPC_URL not set; skipping on-chain enrichment.")
        return df

    w3 = Web3(Web3.HTTPProvider(rpc_url, request_kwargs={"timeout": 30}))

    def fetch_meta(addr: str):
        try:
            c = w3.eth.contract(address=Web3.to_checksum_address(addr), abi=ERC20_ABI_MIN)
            decimals = c.functions.decimals().call()
            try: symbol = c.functions.symbol().call()
            except: symbol = None
            try: name = c.functions.name().call()
            except: name = None
            return decimals, symbol, name
        except Exception:
            return None, None, None

    addrs = (
        df["rewardToken"]
        .dropna()
        .astype(str)
        .str.lower()
        .drop_duplicates()
        .tolist()
    )

    meta = {}
    for a in addrs:
        d, s, n = fetch_meta(a)
        meta[a] = {"decimals": d, "symbol_onchain": s, "name_onchain": n}

    out = df.copy()
    out["rewardToken_lc"] = out["rewardToken"].astype(str).str.lower()
    out["tokenDecimals"] = out["rewardToken_lc"].map(lambda x: meta.get(x, {}).get("decimals"))
    out["symbol_onchain"] = out["rewardToken_lc"].map(lambda x: meta.get(x, {}).get("symbol_onchain"))
    out["name_onchain"] = out["rewardToken_lc"].map(lambda x: meta.get(x, {}).get("name_onchain"))

    # compute amountDecimal if missing and we have decimals
    mask = out["amountDecimal"].isna() & out["amount"].notna() & out["tokenDecimals"].notna()
    out.loc[mask, "amountDecimal"] = (
        pd.to_numeric(out.loc[mask, "amount"], errors="coerce") /
        (10 ** out.loc[mask, "tokenDecimals"].astype(int))
    )

    # prefer on-chain symbol when API didn't provide one
    out["rewardTokenSymbol"] = out["rewardTokenSymbol"].fillna(out["symbol_onchain"])
    return out.drop(columns=["rewardToken_lc"])


In [ ]:
def fetch_opportunities_v4(chain_id: int) -> pd.DataFrame:
    """
    Fetch human-readable opportunity cards (title, protocol, action, APR, TVL, etc.)
    from the canonical v4 endpoint: /v4/opportunities?chainId=<id>
    """
    url = f"{MERKL_API}/v4/opportunities"
    payload = _get_json(url, params={"chainId": chain_id})

    # Response can be a list or a dict with 'opportunities' / 'data' / 'items'
    if isinstance(payload, list):
        items = payload
    elif isinstance(payload, dict):
        if isinstance(payload.get("opportunities"), list):
            items = payload["opportunities"]
        elif isinstance(payload.get("data"), list):
            items = payload["data"]
        elif isinstance(payload.get("items"), list):
            items = payload["items"]
        else:
            raise ValueError(f"Unexpected payload shape keys={list(payload.keys())}")
    else:
        raise ValueError(f"Unexpected payload type: {type(payload)}")

    df_raw = pd.DataFrame(items)

    # helper to pick first present column name
    def pick(*names):
        for n in names:
            if n in df_raw.columns:
                return df_raw[n]
        return None

    df = pd.DataFrame({
        "opportunityId": pick("id","_id","opportunityId"),
        "title": pick("title","name"),
        "protocol": pick("protocol"),
        "action": pick("action"),
        "aprPct": pick("apr","aprPct","apr_percent"),
        "tvlUsd": pick("tvlUsd","tvl_usd","tvl"),
        "dailyRewardsUsd": pick("dailyRewardsUsd","daily_rewards_usd"),
        "rewards": pick("rewards"),
        "campaignIds": pick("campaignIds","campaignIDs"),
        "appLink": pick("appLink","link"),
    })

    # derive token reward/day if rewards block exists
    dr_amt, dr_sym, dr_usd = [], [], []
    for _, row in df.iterrows():
        rewards = row.get("rewards")
        amt = sym = usd = None
        if isinstance(rewards, dict):
            token = rewards.get("token", {}) if isinstance(rewards.get("token"), dict) else {}
            sym = token.get("symbol")
            amt = _safe_float(rewards.get("perDay"))
            usd = _safe_float(rewards.get("perDayUsd"))
        elif isinstance(rewards, list) and rewards:
            a_sum = u_sum = 0.0
            first_sym = None
            for r in rewards:
                tok = r.get("token", {}) if isinstance(r.get("token"), dict) else {}
                if first_sym is None:
                    first_sym = tok.get("symbol")
                a_sum += _safe_float(r.get("perDay")) or 0.0
                u_sum += _safe_float(r.get("perDayUsd")) or 0.0
            sym = first_sym
            amt = a_sum or None
            usd = u_sum or None
        dr_amt.append(amt)
        dr_sym.append(sym)
        dr_usd.append(usd if usd is not None else row.get("dailyRewardsUsd"))

    df["dailyRewardsTokenAmount"] = dr_amt
    df["dailyRewardsTokenSymbol"] = dr_sym
    df["dailyRewardsUsd"] = dr_usd

    df["chainId"] = chain_id
    df["campaignIds"] = df["campaignIds"].apply(
        lambda v: v if isinstance(v, list) else ([v] if isinstance(v, str) and v else [])
    )
    df["appLink"] = df["appLink"].fillna(f"https://app.merkl.xyz/?chain={chain_id}")

    # keep just the normalized columns
    return df[[
        "opportunityId","chainId","title","protocol","action","aprPct","tvlUsd",
        "dailyRewardsUsd","dailyRewardsTokenAmount","dailyRewardsTokenSymbol",
        "campaignIds","appLink"
    ]]


In [7]:
def join_opportunities_with_campaigns(df_opps: pd.DataFrame, df_camps: pd.DataFrame) -> pd.DataFrame:
    df_exploded = df_opps.explode("campaignIds").rename(columns={"campaignIds": "campaignId"})
    df_join = df_exploded.merge(df_camps, how="left", on="campaignId", suffixes=("", "_v3"))

    # authoritative reward metadata from v3
    df_join["amountDecimal"] = pd.to_numeric(df_join["amountDecimal"], errors="coerce")

    cols = [
        "opportunityId","title","protocol","action","aprPct","tvlUsd","dailyRewardsUsd",
        "dailyRewardsTokenAmount","dailyRewardsTokenSymbol","appLink","chainId",
        "campaignId","campaignType","rewardTokenSymbol","rewardToken","amountDecimal",
        "startTimestamp","endTimestamp"
    ]
    for c in cols:
        if c not in df_join.columns:
            df_join[c] = None
    return df_join[cols]


In [8]:
# 1) v3 (always)
df_camps = fetch_campaigns_v3_flat(CHAIN_ID)

# 2) optional on-chain enrichment for decimals/symbol
try:
    df_camps = enrich_with_onchain_token_meta(df_camps, 'https://rpc.plasma.to')
except Exception as e:
    print(f"[warn] on-chain enrichment skipped/failed: {e}")

# save v3
path_v3 = os.path.join(OUT_DIR, "merkl_chain_9745_campaigns.csv")
df_camps.to_csv(path_v3, index=False)
print(f"Saved {path_v3} ({len(df_camps)} rows)")

# 3) v4 (optional) and join
opps_ok = True
try:
    df_opps = fetch_opportunities_v4(CHAIN_ID)
    path_v4 = os.path.join(OUT_DIR, "merkl_chain_9745_opportunities.csv")
    df_opps.to_csv(path_v4, index=False)
    print(f"Saved {path_v4} ({len(df_opps)} rows)")

    df_joined = join_opportunities_with_campaigns(df_opps, df_camps)
    path_joined = os.path.join(OUT_DIR, "merkl_chain_9745_joined.csv")
    df_joined.to_csv(path_joined, index=False)
    print(f"Saved {path_joined} ({len(df_joined)} rows)")
except Exception as e:
    opps_ok = False
    print(f"[warn] v4 opportunities not available: {e}")


Saved outputs/merkl_chain_9745_campaigns.csv (38 rows)
[warn] v4 opportunities not available: v4 opportunities unavailable: 404 Client Error: Not Found for url: https://api.merkl.xyz/v4/public/opportunities?chainId=9745&live=true


In [9]:
print("V3 campaigns:")
display(df_camps.head(10))

if 'df_opps' in globals():
    print("V4 opportunities:")
    display(df_opps.head(10))

if 'df_joined' in globals():
    print("Joined view:")
    display(df_joined.head(10))


V3 campaigns:


,campaignId,rewardToken,chainId,index,creator,campaignType,campaignSubType,amount,startTimestamp,endTimestamp,...,symbolToken1,tick,priceRewardToken,tvl,rewardTokenSymbol,amountDecimal,appLink,tokenDecimals,symbol_onchain,name_onchain
0,0xd04f588ed2148036c11786af5ec5f92e74dac4462f1d...,0x05225a6416EDaeeC7227027E86F7A47D18A06b91,9745,0,0xA9DdD91249DFdd450E81E1c56Ab60E1A62651701,18,0,9700000000,1758034800,1758060000,...,NaN,NaN,NaN,NaN,aglaMerkl,9700.00,https://app.merkl.xyz/?chain=9745#0xd04f588ed2...,6.0,aglaMerkl,aglaMerkl
1,0xaff57ccfe9afc7b7bae6b97e7940c2fc73bcd1f6d203...,0x6100E367285b01F48D07953803A2d8dCA5D19873,9745,3,0x8cd34466193EC736dFc64bDf3892f0321FeE9c35,18,0,2910000000000000000,1758567600,1758571200,...,NaN,NaN,NaN,NaN,WXPL,2.91,https://app.merkl.xyz/?chain=9745#0xaff57ccfe9...,18.0,WXPL,Wrapped XPL
2,0x6b945b754338f44ab6743226a2581f872d833a30ea9b...,0x6100E367285b01F48D07953803A2d8dCA5D19873,9745,2,0xdfA68cD659730E967F0d1213046E4363E424E5b6,18,0,9700000000000000000,1758286800,1758632400,...,NaN,NaN,NaN,NaN,WXPL,9.70,https://app.merkl.xyz/?chain=9745#0x6b945b7543...,18.0,WXPL,Wrapped XPL
3,0x3e0ec6ec92715a069df2bc7c68568e681eae3ec4b662...,0x05225a6416EDaeeC7227027E86F7A47D18A06b91,9745,7,0xA9DdD91249DFdd450E81E1c56Ab60E1A62651701,18,0,10000000000,1758790800,1758794400,...,NaN,NaN,NaN,NaN,aglaMerkl,10000.00,https://app.merkl.xyz/?chain=9745#0x3e0ec6ec92...,6.0,aglaMerkl,aglaMerkl
4,0x18d74ff230386780ad1c3862f13902b4514766994daf...,0x6100E367285b01F48D07953803A2d8dCA5D19873,9745,27,0x8f457a595174f6D2590f3FEd803035935E9a1D57,18,0,81250000000000000000000,1758801600,1759406400,...,NaN,NaN,NaN,NaN,WXPL,81250.00,https://app.merkl.xyz/?chain=9745#0x18d74ff230...,18.0,WXPL,Wrapped XPL
5,0x8f93e29bc9cb1989c0b08c49aceb492954368b907c61...,0x6100E367285b01F48D07953803A2d8dCA5D19873,9745,8,0x8f457a595174f6D2590f3FEd803035935E9a1D57,18,0,12500000000000000000000,1758801600,1759406400,...,NaN,NaN,NaN,NaN,WXPL,12500.00,https://app.merkl.xyz/?chain=9745#0x8f93e29bc9...,18.0,WXPL,Wrapped XPL
6,0x4184741cdd9eb93c1b65af721a98b58f51c88bc4cc97...,0xBa3335588D9403515223F109EdC4eB7269a9Ab5D,1,2603,0x6f378f36899cEB7C6fB7D293aAE1ca86B0Edbf6D,18,0,1620000000000000000000000,1758808800,1759413600,...,NaN,NaN,NaN,NaN,None,NaN,https://app.merkl.xyz/?chain=9745#0x4184741cdd...,NaN,None,None
7,0xdf9d406fdbf3cd394613e54b491126651facbcebd74f...,0x6100E367285b01F48D07953803A2d8dCA5D19873,9745,9,0x8f457a595174f6D2590f3FEd803035935E9a1D57,18,0,46875000000000000000000,1758801600,1759406400,...,NaN,NaN,NaN,NaN,WXPL,46875.00,https://app.merkl.xyz/?chain=9745#0xdf9d406fdb...,18.0,WXPL,Wrapped XPL
8,0x2b5b28273a5070ed78f11f67af6a6faeade104f615df...,0x6100E367285b01F48D07953803A2d8dCA5D19873,9745,20,0x8f457a595174f6D2590f3FEd803035935E9a1D57,18,0,75000000000000000000000,1758801600,1759406400,...,NaN,NaN,NaN,NaN,WXPL,75000.00,https://app.merkl.xyz/?chain=9745#0x2b5b28273a...,18.0,WXPL,Wrapped XPL
9,0x97d5d71daf5d7a9d805a21593b41c5e731d1190a0ecb...,0x6100E367285b01F48D07953803A2d8dCA5D19873,9745,21,0x8f457a595174f6D2590f3FEd803035935E9a1D57,18,0,75000000000000000000000,1758801600,1759406400,...,NaN,NaN,NaN,NaN,WXPL,75000.00,https://app.merkl.xyz/?chain=9745#0x97d5d71daf...,18.0,WXPL,Wrapped XPL
